In [1]:
import json

# Загрузка исходных данных
with open("/Users/margotiamanova/Desktop/google_maps_api/places_analysis/data/places_results.json", "r") as f:
    data = json.load(f)

output = []

for place in data.get("places", []):
    name = place.get("name")
    reviews_raw = place.get("reviews", [])

    reviews = []
    for review in reviews_raw:
        author = review.get("author_name")
        rating = review.get("rating")
        text = review.get("text")

        if author and rating is not None and text:
            reviews.append({
                "author_name": author,
                "rating": rating,
                "text": text.strip()
            })

    if name and reviews:
        output.append({
            "name": name,
            "reviews": reviews
        })

# Сохраняем результат
with open("reports/business_reviews.json", "w") as f:
    json.dump(output, f, indent=2, ensure_ascii=False)


In [5]:
pip install textblob


  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached regex-2024.11.6-cp310-cp310-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 4.6 MB/s eta 0:00:00-:--:--
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp310-cp310-macosx_11_0_arm64.whl (284 kB)
Using cached click-8.2.1-py3-none-any.whl (102 kB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [textblob]4/6 [nltk]b]

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:

import json
from textblob import TextBlob

def analyze_review_sentiment(text):
    """Анализ тональности отзыва с использованием TextBlob"""
    blob = TextBlob(text)
    return blob.sentiment.polarity  # -1 to 1

def compute_investment_score(business):
    reviews = business.get("reviews", [])
    if not reviews:
        return 0

    total_rating = 0
    total_sentiment = 0
    for review in reviews:
        rating = review.get("rating", 0)
        text = review.get("text", "")
        sentiment = analyze_review_sentiment(text)
        total_rating += rating
        total_sentiment += sentiment

    avg_rating = total_rating / len(reviews)
    avg_sentiment = total_sentiment / len(reviews)
    count_reviews = len(reviews)

    # Финальный скор (можно подстроить веса)
    investment_score = (avg_rating * 0.5) + (avg_sentiment * 2) + (count_reviews * 0.05)
    return round(investment_score, 2)

# Загрузка данных
with open("/Users/margotiamanova/Desktop/google_maps_api/places_analysis/data/places_results.json", "r") as f:
    data = json.load(f)

# Создание отчета
results = []
for place in data.get("places", []):
    name = place.get("name")
    reviews = place.get("reviews", [])
    score = compute_investment_score(place)
    results.append({
        "name": name,
        "investment_score": score,
        "reviews_count": len(reviews),
        "rating": place.get("rating")
    })

# Сохранение
with open("reports/investment_scores.json", "w") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("✅ Инвестиционные рейтинги сохранены в reports/investment_scores.json")


✅ Инвестиционные рейтинги сохранены в reports/investment_scores.json
